In [1]:
pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 4.2 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [92]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
import json
import csv

In [93]:
producer = KafkaProducer(bootstrap_servers=['18.117.126.83:9092'], #change ip here
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

# API Data

In [4]:
import requests



# An api key is emailed to you when you sign up to a plan
# Get a free API key at https://api.the-odds-api.com/
API_KEY = 'b799e038d9ee237f501947609f2601ef'

SPORT = 'americanfootball_nfl' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix



In [5]:

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# First get a list of in-season sports
#   The sport 'key' from the response can be used to get odds in the next request
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

sports_response = requests.get('https://api.the-odds-api.com/v4/sports', params={
    'api_key': API_KEY
})


if sports_response.status_code != 200:
    print(f'Failed to get sports: status_code {sports_response.status_code}, response body {sports_response.text}')

else:
    print('List of in season sports:', sports_response.json())




List of in season sports: [{'key': 'americanfootball_ncaaf', 'group': 'American Football', 'title': 'NCAAF', 'description': 'US College Football', 'active': True, 'has_outrights': False}, {'key': 'americanfootball_nfl', 'group': 'American Football', 'title': 'NFL', 'description': 'US Football', 'active': True, 'has_outrights': False}, {'key': 'americanfootball_nfl_super_bowl_winner', 'group': 'American Football', 'title': 'NFL Super Bowl Winner', 'description': 'Super Bowl Winner 2023/2024', 'active': True, 'has_outrights': True}, {'key': 'baseball_mlb', 'group': 'Baseball', 'title': 'MLB', 'description': 'Major League Baseball', 'active': True, 'has_outrights': False}, {'key': 'baseball_mlb_world_series_winner', 'group': 'Baseball', 'title': 'MLB World Series Winner', 'description': 'World Series Winner 2023', 'active': True, 'has_outrights': True}, {'key': 'basketball_euroleague', 'group': 'Basketball', 'title': 'Basketball Euroleague', 'description': 'Basketball Euroleague', 'active

In [6]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
# This will deduct from the usage quota
# The usage quota cost = [number of markets specified] x [number of regions specified]
# For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds', params={
    'api_key': API_KEY,
    'regions': REGIONS,
    'markets': MARKETS,
    'oddsFormat': ODDS_FORMAT,
    'dateFormat': DATE_FORMAT,
})

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 15
[{'id': '5176634f6d30e29b08199d1e0ed6309d', 'sport_key': 'americanfootball_nfl', 'sport_title': 'NFL', 'commence_time': '2023-10-03T00:18:00Z', 'home_team': 'New York Giants', 'away_team': 'Seattle Seahawks', 'bookmakers': [{'key': 'fanduel', 'title': 'FanDuel', 'last_update': '2023-10-02T05:49:51Z', 'markets': [{'key': 'h2h', 'last_update': '2023-10-02T05:49:51Z', 'outcomes': [{'name': 'New York Giants', 'price': 2.08}, {'name': 'Seattle Seahawks', 'price': 1.79}]}]}, {'key': 'betus', 'title': 'BetUS', 'last_update': '2023-10-02T05:50:30Z', 'markets': [{'key': 'h2h', 'last_update': '2023-10-02T05:50:30Z', 'outcomes': [{'name': 'New York Giants', 'price': 2.1}, {'name': 'Seattle Seahawks', 'price': 1.77}]}]}, {'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2023-10-02T05:49:51Z', 'markets': [{'key': 'h2h', 'last_update': '2023-10-02T05:49:51Z', 'outcomes': [{'name': 'New York Giants', 'price': 2.14}, {'name': 'Seattle Seahawks', 'price': 1.74}]}]}, {'ke

In [18]:
odds = odds_response.json()
print(odds)

[{'id': '5176634f6d30e29b08199d1e0ed6309d', 'sport_key': 'americanfootball_nfl', 'sport_title': 'NFL', 'commence_time': '2023-10-03T00:18:00Z', 'home_team': 'New York Giants', 'away_team': 'Seattle Seahawks', 'bookmakers': [{'key': 'fanduel', 'title': 'FanDuel', 'last_update': '2023-10-02T05:49:51Z', 'markets': [{'key': 'h2h', 'last_update': '2023-10-02T05:49:51Z', 'outcomes': [{'name': 'New York Giants', 'price': 2.08}, {'name': 'Seattle Seahawks', 'price': 1.79}]}]}, {'key': 'betus', 'title': 'BetUS', 'last_update': '2023-10-02T05:50:30Z', 'markets': [{'key': 'h2h', 'last_update': '2023-10-02T05:50:30Z', 'outcomes': [{'name': 'New York Giants', 'price': 2.1}, {'name': 'Seattle Seahawks', 'price': 1.77}]}]}, {'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2023-10-02T05:49:51Z', 'markets': [{'key': 'h2h', 'last_update': '2023-10-02T05:49:51Z', 'outcomes': [{'name': 'New York Giants', 'price': 2.14}, {'name': 'Seattle Seahawks', 'price': 1.74}]}]}, {'key': 'lowvig', 'title'

In [69]:
data = []
bookmaker = 2  #Draftkings
header = ['DATE', 'HOME_TEAM','HOME_ODDS', 'AWAY_TEAM', 'AWAY_ODDS']

In [70]:
# for obj in odds:
#         print(obj['bookmakers'][2]['markets'][0]['outcomes'])
#         #print(obj)
#         #print(list(obj.keys()))
#         print("*")
#         print("*")
#         print("*")
#         print("*")
#         print("*")
#         print("*")

In [71]:

for obj in odds:
    date = obj['commence_time']
    home = obj['bookmakers'][bookmaker]['markets'][0]['outcomes'][0]['name']
    home_odds = obj['bookmakers'][bookmaker]['markets'][0]['outcomes'][0]['price']
    away = obj['bookmakers'][bookmaker]['markets'][0]['outcomes'][1]['name']
    away_odds = obj['bookmakers'][bookmaker]['markets'][0]['outcomes'][1]['price']
    
    listData = [date, home, home_odds, away, away_odds]
    data.append(listData)
    
with open('games.csv', 'w', encoding='UTF8', newline='') as f:
    csvWriter = csv.writer(f)
    
    csvWriter.writerow(header)
    csvWriter.writerows(data)
    

# Producer to Consumer

In [89]:
df = pd.read_csv("/Users/anish/games.csv")

size = df.shape[0]

In [95]:
for i in range(size):
    game_odd = df.to_dict(orient="records")[i]
    producer.send('h2h_odds', value=game_odd)

In [94]:
producer.flush()